In [1]:
#Created on Wed Apr 21 23:01:41 2021

#@author: Thiago Novaes

In [6]:
# include

import os
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

In [7]:
#user

dir_HS = "HS serie"   #diretorio local com os HS
dir_B = "B serie"     #diretorio local com os B

interpol = "linear"      #tipo de interpolação desejado


v = 11.9       #velocidade

D_min = 2.5   #diametro minimo 
D_max = 2.5   #diametro maximo
D_num = 1   #numero de subdivisoes

n_min = 200/60  #rotação minima
n_max = 400/60  #rotação maxima
n_num = 11       #numero de subdivisoes

helices = 4
T = 430800      #força requerida
tol = 0.5       #tolerancia da força requerida em %; float("inf")
rho = 1025      #densidade do fluido

serie = 'HS' # B or HS

h = 3.4 # altura do helice

Rn = 5E9           #B serie only
corr_reynolds = False #B serie only 

output = 'final.csv' #path do output
sigma = 1.81
eixo = 0.99
T = T/helices

In [8]:
# function

sigmas = [0.25,0.3,0.4,0.5,0.6,0.75,1,2.5]
sigmas_str = ["0,25","0,30","0,40","0,50","0,60","0,75","1,00","2,50"]

def lim_inter(sigma):
    for i in range(len(sigmas)):
        if sigma < sigmas[i]:
            return i-1, i 
a,b = lim_inter(sigma)

def J(v,n,D):
    return v/(n*D)

def eta(j,Kt,Kq):
    return j*Kt/(2*np.pi*Kq)

def Trust(Kt,n,D):
    return Kt*(rho*n**2*D**4)

def Torque(Kq,n,D):
    return Kq*(rho*n**2*D**5)

def cavitation(D,u,t,v,n,h,T,rho):
    Ap = np.pi*D**2/4*u*np.cos(np.arctan(t))           
    Vs2 = v**2+(np.pi*n*0.7*D)**2
    sigma = (100000 + 10000*h)/(rho/2*Vs2)
    cav = T/Ap/(rho/2*Vs2)/(0.42*sigma**(0.7))
    return cav, sigma

def main(v,D_min,D_max,D_num,n_min,n_max,n_num,T,tol,serie,output,interpol = "linear",h = 0, Rn = 0,corr_reynolds = False,sigma =0):
    out = pd.DataFrame(index=range(0),columns=range(15))
    out.columns = ["Blades","BAR","P/D", "sigma","D","n", "J", "Kt", "Kq", "Trust", "Q", "Eta","THP","DHP","BHP"]
    for D in np.linspace(D_min,D_max,D_num):
        for n in np.linspace(n_min,n_max,n_num):
            j = J(v,n,D)
            if serie == 'HS':
                out = HSsigma(j,v,D,n,T,tol,interpol,out,sigma)
            elif serie == 'B' and 0 <= j <= 1.6:
                out = B(j,v,D,n,T,tol,interpol,out,h,Rn,corr_reynolds)
            else:
                pass
    out.sort_values(by=["BHP"],ascending=True, inplace=True)
    out.to_csv(output,sep=";")
    return out
                
def HS(j,v,D,n,T,tol,interpol,out):
    for filename in os.listdir(dir_HS):
        df = pd.read_csv(os.path.join(dir_HS,filename))
        if df.head(1)["J"].values <= j <= df.tail(1)["J"].values:
            Kt = interp1d(df["J"], df["Kt"],kind =interpol)(j)
            Kq = interp1d(df["J"], df["Kq"],kind =interpol)(j)
            Eta = interp1d(df["J"], df["Eta"],kind =interpol)(j)
            out = validation_HS(j,v,D,n,T,tol,Kt,Kq,Eta,out,filename)
        else:
            pass
    return out

def HSsigma(j,v,D,n,T,tol,interpol,out,sigma):
    for filename1 in os.listdir(dir_HS):
        if filename1[13:17] == sigmas_str[a]:
            for filename2 in os.listdir(dir_HS):
                if filename2[:17] == filename1[:13] + sigmas_str[b]:
                    df1 = pd.read_csv(os.path.join(dir_HS,filename1))
                    df2 = pd.read_csv(os.path.join(dir_HS,filename2))
                    if df1.head(1)["J"].values <= j <= df1.tail(1)["J"].values and df2.head(1)["J"].values <= j <= df2.tail(1)["J"].values:
                        Kt1 = interp1d(df1["J"], df1["Kt"],kind =interpol)(j)
                        Kq1 = interp1d(df1["J"], df1["Kq"],kind =interpol)(j)
                        Eta1 = interp1d(df1["J"], df1["Eta"],kind =interpol)(j)
                        
                        Kt2 = interp1d(df2["J"], df2["Kt"],kind =interpol)(j)
                        Kq2 = interp1d(df2["J"], df2["Kq"],kind =interpol)(j)
                        Eta2 = interp1d(df2["J"], df2["Eta"],kind =interpol)(j)
                        
                        Kt = interp1d(sigmas[a:a+2], [Kt1,Kt2])(sigma)
                        Kq = interp1d(sigmas[a:a+2], [Kq1,Kq2])(sigma)
                        Eta = interp1d(sigmas[a:a+2], [Eta1,Eta2])(sigma)
                        out = validation_HS(j,v,D,n,T,tol,Kt,Kq,Eta,out,filename1,sigma)
                    else:
                        pass
    return out

def validation_HS(j,v,D,n,T,tol,Kt,Kq,Eta,out,filename,sigma):
    trust = Trust(Kt,n,D)
    Q = Torque(Kq,n,D)
    THP = v*trust
    DHP = THP/Eta
    BHP = DHP/eixo
    if T <= trust <=T*(1+tol):
        blades, bar , pd , lixo, ext = filename.split("-")
        new_row = {"Blades": blades,"BAR":bar,"P/D":pd,"sigma":sigma,"D":D,"n":n, "J":j, "Kt":Kt,"Kq":Kq, "Trust":trust,"Q":Q ,"Eta":Eta,"THP":THP,"DHP":DHP,"BHP":BHP}
        out = out.append(new_row, ignore_index=True)
    else:
        pass
    return out

def B(j,v,D,n,T,tol,interpol,out,h,Rn,corr_reynolds):
    df_kt = pd.read_csv(os.path.join(dir_B,'kt.csv'))
    df_kq = pd.read_csv(os.path.join(dir_B,'kq.csv'))
    df_dkt = pd.read_csv(os.path.join(dir_B,'dkt.csv'))
    df_dkq = pd.read_csv(os.path.join(dir_B,'dkq.csv'))
    for t in np.linspace(0.5,1.4,10):
        for u in np.linspace(0.3,1.05,16):
            for z in np.linspace(2,7,6):
                Kt = (df_kt["coef"]*j**df_kt["s"]*t**df_kt["t"]*u**df_kt["u"]*z**df_kt["v"]).sum() 
                Kq = (df_kq["coef"]*j**df_kq["s"]*t**df_kq["t"]*u**df_kq["u"]*z**df_kq["v"]).sum()
                if corr_reynolds == True and Rn > 2E6:
                    R = np.log10(Rn)-0.301
                    Kt += (df_dkt["coef"]*R**df_dkt["r"]*j**df_dkt["s"]*t**df_dkt["t"]*u**df_dkt["u"]*z**df_dkt["v"]).sum()
                    Kq += (df_dkq["coef"]*R**df_dkq["r"]*j**df_dkq["s"]*t**df_dkq["t"]*u**df_dkq["u"]*z**df_dkq["v"]).sum()
                Eta = eta(j,Kt,Kq)
                out = validation_B(j,v,D,n,T,tol,Kt,Kq,Eta,out,t,u,z,h)
    return out

def validation_B(j,v,D,n,T,tol,Kt,Kq,Eta,out,t,u,z,h):
    cav, sigma = cavitation(D,u,t,v,n,h,T,rho)
    trust = Trust(Kt,n,D)
    Q = Torque(Kq,n,D)
    THP = v*trust
    DHP = THP/Eta
    BHP = DHP/eixo
    if T <= trust <=T*(1+tol) and Kt > 0 and Kq > 0 and cav < 1:
        new_row = {"Blades": z,"BAR":u,"P/D":t,"sigma":sigma,"D":D,"n":n, "J":j, "Kt":Kt,"Kq":Kq, "Trust":trust,"Q":Q,"Eta":Eta,"THP":THP,"DHP":DHP,"BHP":BHP}
        out = out.append(new_row, ignore_index=True)
    else:
        pass
    return out

In [9]:
main(v,D_min,D_max,D_num,n_min,n_max,n_num,T,tol,serie,output,interpol,h,Rn,corr_reynolds,sigma)

,Blades,BAR,P/D,sigma,D,n,J,Kt,Kq,Trust,Q,Eta,THP,DHP,BHP
6,A3,"0,48","1,26",1.81,2.5,4.333333,1.098462,0.14921800000000016,0.03434695384615387,112188.750217,64558.930990,0.7590212307692307,1.335046e+06,1.758905e+06,1.776672e+06
10,A3,"0,48","1,05",1.81,2.5,5.000000,0.952000,0.11605759999999996,0.024143039999999994,116170.937500,60416.542969,0.724808,1.382434e+06,1.907311e+06,1.926577e+06
2,A3,"0,48","1,67",1.81,2.5,3.666667,1.298182,0.2310630181818182,0.06556930909090909,124381.904644,88240.381510,0.7278396363636364,1.480145e+06,2.033614e+06,2.054155e+06
8,A3,"0,95","1,24",1.81,2.5,4.333333,1.098462,0.15542676923076945,0.040033907692307726,116856.779948,75248.195095,0.6785233846153846,1.390596e+06,2.049444e+06,2.070145e+06
7,A3,"0,71","1,25",1.81,2.5,4.333333,1.098462,0.16600092307692324,0.04020935384615387,124806.900608,75577.965712,0.7219236923076922,1.485202e+06,2.057284e+06,2.078065e+06
3,A3,"0,71","1,66",1.81,2.5,3.666667,1.298182,0.2552739636363637,0.07425312727272729,137414.728038,99926.693902,0.710434909090909,1.635235e+06,2.301738e+06,2.324988e+06
4,A3,"0,95","1,65",1.81,2.5,3.666667,1.298182,0.2583609818181819,0.07636894545454548,139076.479036,102774.071832,0.6985872727272727,1.655010e+06,2.369081e+06,2.393011e+06
0,A3,"0,48","2,08",1.81,2.5,3.333333,1.428000,0.31438768,0.10596216,139864.310764,117850.709635,0.6740328,1.664385e+06,2.469294e+06,2.494237e+06
12,A3,"0,95","1,04",1.81,2.5,5.000000,0.952000,0.13462639999999995,0.03168343999999999,134757.871094,79285.952148,0.6431696,1.603619e+06,2.493306e+06,2.518491e+06
9,A3,"0,48","1,26",1.81,2.5,4.666667,1.020000,0.175234,0.0389748,152797.355035,84961.342448,0.72882,1.818289e+06,2.494839e+06,2.520039e+06
